In [30]:
# 2019 Wes Smith

## import modules ##

from arcgis.gis import GIS
from arcgis.gis import Group

import pandas as pd
import datetime
import os
import json

In [31]:
## Set Global Vars ##

# Timestamp
now = datetime.datetime.now()
time_str = str(now.strftime("%x"))
date_stamp = time_str.replace("/","")

# String for AGO queries 
username = "WESSMITH.BC"
group_name = "Omineca ESI - Developer"
searchkeyword = None

# Path and file name for XLSX output 
directory = r"\\SFP.IDIR.BCGOV\U164\WESSMITH$\!Python\TESTING"
filename = "AGO_ItemsList"
out_f = os.path.join(directory, filename + date_stamp + ".xlsx")

# Item description strings
description_str = ""
summary_str = ""
terms_of_use_str = ""
credits_str = ""
#credits_str = "Muskwa-Kechika Management Area Executive Board"




In [33]:
## Log in to AGO ##
agol = GIS(username=username)

Enter password: ········


In [34]:
## Query content based on title ##

# query groups
ago_groups = agol.groups.search('title:"' + group_name + '"')

# Hold group object
group = ago_groups[0]

## Check
#group

In [35]:
# Get list of item objects as a list
items = group.content()

## Count retrieved item objects ##
items_count = len(items)
items_count

## Check
#items[0]

In [42]:
## Prepare lists of item details ##

# list vars
title = []
itemtype_code = []
itemid = []
url = []
owner = []
itemtype = []
extent = []
description = []
summary = []
tags = []
termofuse = []
credit = []
size = []
modified_date = []
counter = 0

# Loops through item objects append data to lists
for item in items:
    #if (searchkeyword in item.tags) or (searchkeyword in item.title)\
    #        or (searchkeyword + " " in item.tags) or (" " + searchkeyword + " " in item.title):
        
    # Title
    title.append(item.title)

    # Item type code
    item_code = ""
    type_list = ["StoryMap", "Web Mapping Application"]
    level_list = ["L1", "L2", "L3", "L4", "L5", "L6"]
    last_sp = item.title.rfind(" ", 0, len(item.title))
    if item.type not in type_list:
        if item.type == "Feature Service":
            type_code = "FS"
        elif item.type == "Insights Workbook":
            type_code = "IW"
        elif item.type == "Service Definition":
            type_code = "SD"
        elif item.type == "Code Attachment":
            type_code = "CODE"
        elif item.type == "Web Map":
            type_code = "WM"
        else:
            type_code = "ERR"
    elif item.type in type_list:
        if item.title[last_sp:].strip(" ") in level_list:
            new_title = item.title[:last_sp].strip(" ")
            new_last_sp = new_title.rfind(" ", 0, len(new_title))
            type_code = new_title[new_last_sp:].strip(" ")
        else:
            new_title = item.title[:last_sp].strip(" ")
            type_code = new_title[last_sp:].strip(" ")
        if type_code in ["", " "]:
            type_code = "T ERR"
    itemtype_code.append(type_code)

    # Item ID
    itemid.append(item.itemid)

    # URL
    if item.url is None:
        url.append("https://governmentofbc.maps.arcgis.com/home/item.html?id="+item.itemid)
    else:
        url.append(item.url)

    # Type
    itemtype.append(item.type)

    # Owner
    owner.append(item.owner)

    # Description
    if item.description is not None:
        testhtml = item.description
        description.append(testhtml)
    else:
        description.append(None)

    # Summary
    summary.append(item.snippet)

    # Tags
    tag_str = str(item.tags).strip(" ")
    x_chars = ["[", "]", "'", '"']
    for char in x_chars:
        tag_str = tag_str.replace(char, "")
    #print("{}\n{}\n".format(item.tags, tag_str))
    tags.append(tag_str)
    # Terms of use
    termofuse.append(item.licenseInfo)

    # Credits
    credit.append(item.accessInformation)

    # Size on AGO
    size.append(item.size)

    # Date Modified
    modified_date.append(datetime.datetime.fromtimestamp(int(item.modified/1000)).strftime('%Y%m%d'))

    # Extent
    #extent_var = "None"
    #if item.type == "Web Map":
    #extent_var = json.dumps(item.extent)
    #extent.append(extent_var)

            

In [40]:
## Create a data frame ##
df = pd.DataFrame()

In [43]:
## Fill the data frame ##

df["Title"] = title
df["Type Code"] = itemtype_code
df['Item ID'] = itemid
df["Item Type"] = itemtype
df["Owner"] = owner
#df["Extent"] = extent
df['URL'] = url
df["Summary"] = summary
df["Description"] = description
df["Tags"] = tags
df["Term of Use"] = termofuse
df["Credits"] = credit
df["Size on AGO"] = size
df["Modified Date"] = modified_date
df["Public"] = None
df["Data Catalog Name"] = None

df

,Title,Type Code,Item ID,Item Type,Owner,URL,Summary,Description,Tags,Term of Use,Credits,Size on AGO,Modified Date,Public,Data Catalog Name
0,BEC Zones 2050 ESI Study Area,FS,568f16b5366a4a229f0ec1d63b64f607,Feature Service,WESSMITH.BC,https://services6.arcgis.com/ubm4tcTYICKBpist/...,Forecasted BEC Zones in 2050 based on current ...,nan,ESI,nan,Omineca ESI Project Team,1417216,20200206,None,None
1,Consolidated_Roads,FS,f2843d14fc324cc899f286fc3e5fa974,Feature Service,WESSMITH.BC,https://services6.arcgis.com/ubm4tcTYICKBpist/...,nan,nan,"BC, BC Government, sourceId-f29bc4a21ead469f9b...",nan,Omineca ESI Project Team,32030720,20200206,None,None
2,DRAFT OminecaESI Welcome NSMT L2,NSMT,0dc3466467114a228aaf68474fc4424a,StoryMap,WESSMITH.BC,https://governmentofbc.maps.arcgis.com/home/it...,To the Omineca Environmental Stewardship Initi...,nan,"Omineca Region, CSTC Management Area, ESI",nan,Omineca ESI Project Team,8012,20200206,None,None
3,ESI Biodiversity BMA Poly Dissolve,SD,1603568af2674859b0a2be31c97af029,Service Definition,WESSMITH.BC,https://governmentofbc.maps.arcgis.com/home/it...,ESI BMA polygons dissolved by gridcode,nan,ESI,nan,Omineca ESI Project Team,6409479,20200206,None,None
4,ESI_BioDiv_Logged_Riparian,FS,6261c6852cc64ac695dcb47bf6b3fd6a,Feature Service,WESSMITH.BC,https://services6.arcgis.com/ubm4tcTYICKBpist/...,nan,nan,"CSFN Management Area, BC, ESI, Omineca Regio...",nan,Omineca ESI Project Team,5677056,20200206,None,None
5,ESI_FTA_PlannedPermittedBlocks2,FS,60365adfa5184936b55c2ada5f6aec0d,Feature Service,WESSMITH.BC,https://services6.arcgis.com/ubm4tcTYICKBpist/...,nan,nan,"Blocks, CSFN Management Area, ESI, Omineca Re...",nan,Omineca ESI Project Team,20258816,20200206,None,None
6,Omineca ESI Biodiversity 2017 BMA Poly Dissolv...,FS,6d3c7be7b1104565ae0e314a5454352d,Feature Service,WESSMITH.BC,https://services6.arcgis.com/ubm4tcTYICKBpist/...,BMAs by dissolved by gridcode and ESI Quad,nan,ESI,nan,Omineca ESI Project Team,64290816,20200206,None,None
7,Omineca ESI Biodiversity BMA 2017 Poly Dissolv...,FS,387da3c154514e9d8bb893be1ece30b5,Feature Service,WESSMITH.BC,https://services6.arcgis.com/ubm4tcTYICKBpist/...,Omineca ESI Biodiversity BMA 2017 Poly Dissolv...,nan,ESI,nan,Omineca ESI Project Team,67870720,20200206,None,None
8,Omineca ESI Biodiversity BMA 2017 Poly Dissolv...,SD,a4de7b77f581414686537aaffe9ac433,Service Definition,WESSMITH.BC,https://governmentofbc.maps.arcgis.com/home/it...,Omineca ESI Biodiversity BMA 2017 Poly Dissolv...,nan,ESI,nan,Omineca ESI Project Team,4072589,20200206,None,None
9,Omineca ESI BMAs _Oct 31_ 2017_,SD,9a7787553c4141e78453b2a2dd157c5b,Service Definition,WESSMITH.BC,https://governmentofbc.maps.arcgis.com/home/it...,BMAs by dissolved by gridcode and ESI Quad,nan,ESI,nan,Omineca ESI Project Team,7131018,20200206,None,None


In [44]:
## Save the dataframe as XLSX file ##

# file does not exist
if not os.path.exists(out_f):
    df.to_excel(out_f)
# file exists
else:
    print("A file with the same name alreayd exists.  Would you like to overwrite it?")
    overwrite = input("type y/n > ")
    print(overwrite)
    if overwrite == "y":
        os.remove(out_f)
        df.to_excel(out_f)
        message = "data was overwritten"
    else:
        message = "data was not overwritten"
    print(message)
    
print("Script is complete...")
os.startfile(out_f, 'open')

A file with the same name alreayd exists.  Would you like to overwrite it?
type y/n > y
y
data was overwritten
Script is complete...
